# Main Notebook

File to perform experiments

## Imports

In [1]:
import os
import numpy as np
import gpflow
import matplotlib.pyplot as plt
import pandas as pd
import time
from tqdm import tqdm
import tensorflow as tf

from models.GaussianProcess import GaussianProcess
from acquisition_functions.UseMO import usemo_acq
from acquisition_functions.MES import mes_acq, basic_mes_acq
from acquisition_functions.PESMO import pesmo_acq
from acquisition_functions.MESMO import mesmo_acq
from acquisition_functions.SingleObjective import simulated_mes_bins
from arguments.arguments import MainArguments

from MOObenchmark import MOOackley, MOOexponential, MOOquadratic_ackley, MOOnnHeartNeuronsF1
from utils.calc_pareto import get_pareto_undominated_by, getSetfromFront

from models.MOOEvaluationProblem import MOOEvaluationProblem

from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.factory import get_termination
from pymoo.optimize import minimize

2023-06-18 19:30:34.407006: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-18 19:30:34.447070: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-18 19:30:34.447521: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-18 19:30:35.030848: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Algorithm Arguments

In [2]:
d = 5 # input dimensions
    
seed = 0
np.random.seed(seed)

## iterations of the main algorithm
total_iter = 40
## iterations taken randomly at the beggining
initial_iter = 1

## bound of the inputs definition
### with the current implementations all the input dimensions must have the same bounds, 
### but this can be changed
lower_bound = 10
upper_bound = 1000

lowerBounds = [lower_bound]*d
upperBounds = [upper_bound]*d


## Evaluation

In [3]:
## Evaluation function definition
O = 1
C = 0

def evaluation(x):
    return MOOnnHeartNeuronsF1(x)

This cell below gets the real pareto in two ways: with a grid and with a cheap optimizer (genetic algorithm) and plots both of them. These results are used in test problems as the actual pareto front. Although, to obtain these paretos many evaluations of the function are needed, so it has no sense running it in a costly black box problem.

In addition this is only prepared to plot a d=1, O=2 problem. We could visualize any problem, but the visualization get a lot more abstract and more plots are needed. If this is interesting I could do it.

Therefore I set the code below as text.

N = 1_001
X = np.linspace(lower_bound,upper_bound,N)
Z = np.zeros((N,2))

problem = MOOEvaluationProblem(evaluation, O=O, d=d, lowerBounds=lower_bound, upperBounds=upper_bound)
algorithm = NSGA2()
res = minimize( problem, 
                algorithm,
                termination = get_termination("n_gen",100))

real_pareto = res.F[np.argsort(res.F[:,0])]

for i in range(N):
    Z[i]=evaluation(X[i])

fig, axs = plt.subplots(1,3, figsize=(15,4))

axs[0].plot(X, Z[:,0], 'b')
axs[0].plot(X, Z[:,1], 'k')
axs[0].plot(res.X, res.F[:,0], 'xr', markersize=5)
axs[0].plot(res.X, res.F[:,1], 'xr', markersize=5)

axs[1].plot(np.reshape(Z,(-1,2))[:,0], np.reshape(Z,(-1,2))[:,1], 'kx')
axs[1].plot(res.F[:,0], res.F[:,1], 'rx')
res
axs[2].plot(res.F[:,0], res.F[:,1], 'x')
plt.show()

In [4]:
def random_acq(GP: GaussianProcess, **kwargs):
    while True:
        x_rand = np.random.uniform(GP.lowerBounds, GP.upperBounds, GP.d)
        if GP.X is None or not x_rand in GP.X:
            break
    return x_rand, 0

## N experiments

In [5]:
savename = "MOONNHeart"

### GPs Initialization
GP = GaussianProcess(O, C, d, lowerBounds, upperBounds, noise_variance=2e-6)

#### Initial samples, at least 1
for l in range(initial_iter):
    ## Get random evaluation point
    while True:
        x_rand = np.random.uniform(lowerBounds[0], upperBounds[0], d)
        if GP.X is None or not x_rand in GP.X:
            break
    ## EVALUATION OF THE OUTSIDE FUNCTION
    
    y_rand = evaluation(x_rand)
    GP.X = np.array([x_rand], dtype=float)
    GP.Y = np.array([y_rand], dtype=float)

GP.updateGP()
GP.optimizeKernel()
if False:
    GP.plotSamples()

for l in range(total_iter):

    ## For all acq functions but usemo
    x_best, acq_best = simulated_mes_bins(GP)
    ## For usemo
    #x_best, acq_best = usemo_acq(GP, function = "ei", showplots = False)

    ## EVALUATION OF THE OUTSIDE FUNCTION
    y_best = evaluation(x_best)

    ## UPDATE
    GP.addSample(x_best,y_best)     ## Add new sample to the model
    GP.updateGP()                   ## Update data on the GP regressor
    GP.optimizeKernel()             ## Optimize kernel hyperparameters

    print("Iteration=", l, " x_best=", x_best, "y=", y_best)
    

C = [553.32536889 718.03747271 606.73574231 549.43435117 429.41825135]
Epoch 1/6
1210/1210 [==============================] - 4s 3ms/step - loss: 0.5563 - val_loss: 0.5091
Epoch 2/6
1210/1210 [==============================] - 4s 3ms/step - loss: 0.5139 - val_loss: 0.5153
Epoch 3/6
1210/1210 [==============================] - 4s 3ms/step - loss: 0.5122 - val_loss: 0.4898
Epoch 4/6
1210/1210 [==============================] - 4s 3ms/step - loss: 0.5070 - val_loss: 0.4929
Epoch 5/6
1210/1210 [==============================] - 4s 3ms/step - loss: 0.5069 - val_loss: 0.4911
Epoch 6/6
135/135 [==============================] - 0s 794us/step
-0.75 -0.8368200836820083 0.7910349373764007
C = [269.1015625 586.2109375 539.8046875 199.4921875 222.6953125]
Epoch 1/6
1210/1210 [==============================] - 3s 2ms/step - loss: 0.5320 - val_loss: 0.4818
Epoch 2/6
1210/1210 [==============================] - 2s 2ms/step - loss: 0.5027 - val_loss: 0.4787
Epoch 3/6
1210/1210 [=======================

1210/1210 [==============================] - 2s 1ms/step - loss: 0.5318 - val_loss: 0.4893
Epoch 2/6
1210/1210 [==============================] - 2s 1ms/step - loss: 0.5046 - val_loss: 0.4879
Epoch 3/6
1210/1210 [==============================] - 2s 1ms/step - loss: 0.5031 - val_loss: 0.4831
Epoch 4/6
1210/1210 [==============================] - 2s 1ms/step - loss: 0.5012 - val_loss: 0.4804
Epoch 5/6
1210/1210 [==============================] - 2s 1ms/step - loss: 0.4986 - val_loss: 0.4888
Epoch 6/6
135/135 [==============================] - 0s 657us/step
-0.7401861594496155 -0.8503021850302185 0.791432280398096
Current x =  [[553.32536889 718.03747271 606.73574231 549.43435117 429.41825135]
 [269.1015625  586.2109375  539.8046875  199.4921875  222.6953125 ]
 [270.55175781 958.91113281 364.33105469 592.49511719 673.70605469]
 [865.13183594 673.70605469 775.21972656 311.15722656 908.63769531]
 [242.03125     87.34375    798.90625    149.21875    149.21875   ]
 [799.63134766 336.05224609

1210/1210 [==============================] - 1s 1ms/step - loss: 0.5107 - val_loss: 0.4801
Epoch 4/6
1210/1210 [==============================] - 1s 1ms/step - loss: 0.5138 - val_loss: 0.4810
Epoch 5/6
1210/1210 [==============================] - 1s 1ms/step - loss: 0.5128 - val_loss: 0.4836
Epoch 6/6
135/135 [==============================] - 0s 595us/step
-0.7452790600083928 -0.8256624825662483 0.7834142037935599
Current x =  [[553.32536889 718.03747271 606.73574231 549.43435117 429.41825135]
 [269.1015625  586.2109375  539.8046875  199.4921875  222.6953125 ]
 [270.55175781 958.91113281 364.33105469 592.49511719 673.70605469]
 [865.13183594 673.70605469 775.21972656 311.15722656 908.63769531]
 [242.03125     87.34375    798.90625    149.21875    149.21875   ]
 [799.63134766 336.05224609 773.04443359 703.43505859 162.99560547]
 [504.27490234 202.15087891 901.14501953  65.83251953 523.12744141]
 [100.15380859 354.90478516 779.32861328 806.39892578 273.69384766]
 [253.87451172 698.11767

Iteration= 15  x_best= [302.93945312 370.61523438 743.79882812 194.65820312 668.38867188] y= [-0.77960677]
C = [900.66162109 714.55322266 110.30517578 686.51611328 602.40478516]
Epoch 1/6
1210/1210 [==============================] - 2s 2ms/step - loss: 0.5510 - val_loss: 0.4938
Epoch 2/6
1210/1210 [==============================] - 2s 2ms/step - loss: 0.5242 - val_loss: 0.4940
Epoch 3/6
1210/1210 [==============================] - 2s 2ms/step - loss: 0.5187 - val_loss: 0.5013
Epoch 4/6
1210/1210 [==============================] - 2s 2ms/step - loss: 0.5135 - val_loss: 0.4973
Epoch 5/6
1210/1210 [==============================] - 2s 2ms/step - loss: 0.5162 - val_loss: 0.4895
Epoch 6/6
135/135 [==============================] - 0s 667us/step
-0.7005772005772006 -0.902835890283589 0.7889498273410521
Current x =  [[553.32536889 718.03747271 606.73574231 549.43435117 429.41825135]
 [269.1015625  586.2109375  539.8046875  199.4921875  222.6953125 ]
 [270.55175781 958.91113281 364.33105469 59

Iteration= 19  x_best= [819.20898438 565.90820312 548.50585938 498.23242188 755.40039062] y= [-0.79112439]
C = [ 72.35839844 321.79199219 840.96191406  52.05566406  77.19238281]
Epoch 1/6
1210/1210 [==============================] - 2s 1ms/step - loss: 0.5457 - val_loss: 0.5017
Epoch 2/6
1210/1210 [==============================] - 2s 1ms/step - loss: 0.5009 - val_loss: 0.4819
Epoch 3/6
1210/1210 [==============================] - 2s 1ms/step - loss: 0.4945 - val_loss: 0.4800
Epoch 4/6
1210/1210 [==============================] - 2s 1ms/step - loss: 0.4913 - val_loss: 0.4799
Epoch 5/6
1210/1210 [==============================] - 2s 1ms/step - loss: 0.4905 - val_loss: 0.4765
Epoch 6/6
135/135 [==============================] - 0s 637us/step
-0.7404985696771557 -0.8423988842398884 0.788168769030013
Current x =  [[553.32536889 718.03747271 606.73574231 549.43435117 429.41825135]
 [269.1015625  586.2109375  539.8046875  199.4921875  222.6953125 ]
 [270.55175781 958.91113281 364.33105469 59

...

135/135 [==============================] - 0s 652us/step
-0.7587119541243935 -0.799628079962808 0.7786328655500226
Current x =  [[553.32536889 718.03747271 606.73574231 549.43435117 429.41825135]
 [269.1015625  586.2109375  539.8046875  199.4921875  222.6953125 ]
 [270.55175781 958.91113281 364.33105469 592.49511719 673.70605469]
 [865.13183594 673.70605469 775.21972656 311.15722656 908.63769531]
 [242.03125     87.34375    798.90625    149.21875    149.21875   ]
 [799.63134766 336.05224609 773.04443359 703.43505859 162.99560547]
 [504.27490234 202.15087891 901.14501953  65.83251953 523.12744141]
 [100.15380859 354.90478516 779.32861328 806.39892578 273.69384766]
 [253.87451172 698.11767578 979.45556641 909.84619141 141.24267578]
 [458.59375    365.78125    891.71875    860.78125    118.28125   ]
 [143.90136719 172.90527344 908.63769531  23.05175781 148.73535156]
 [690.14160156 821.62597656  68.49121094 478.41308594 161.30371094]
 [ 70.18310547 386.80908203 265.95947266 517.32666016 91

1210/1210 [==============================] - 2s 2ms/step - loss: 0.5185 - val_loss: 0.4862
Epoch 3/6
1210/1210 [==============================] - 2s 2ms/step - loss: 0.5156 - val_loss: 0.4939
Epoch 4/6
1210/1210 [==============================] - 2s 2ms/step - loss: 0.5125 - val_loss: 0.4998
Epoch 5/6
1210/1210 [==============================] - 2s 2ms/step - loss: 0.5134 - val_loss: 0.5195
Epoch 6/6
135/135 [==============================] - 0s 671us/step
-0.7615720524017467 -0.8107856810785681 0.7854086917360955
Current x =  [[553.32536889 718.03747271 606.73574231 549.43435117 429.41825135]
 [269.1015625  586.2109375  539.8046875  199.4921875  222.6953125 ]
 [270.55175781 958.91113281 364.33105469 592.49511719 673.70605469]
 [865.13183594 673.70605469 775.21972656 311.15722656 908.63769531]
 [242.03125     87.34375    798.90625    149.21875    149.21875   ]
 [799.63134766 336.05224609 773.04443359 703.43505859 162.99560547]
 [504.27490234 202.15087891 901.14501953  65.83251953 523.1

Iteration= 31  x_best= [193.20800781 819.69238281 410.73730469 143.90136719 998.54980469] y= [-0.79013414]
C = [964.47021484 778.36181641 263.05908203 154.77783203 944.65087891]
Epoch 1/6
1210/1210 [==============================] - 3s 2ms/step - loss: 0.5544 - val_loss: 0.4993
Epoch 2/6
1210/1210 [==============================] - 3s 2ms/step - loss: 0.5204 - val_loss: 0.4977
Epoch 3/6
1210/1210 [==============================] - 3s 2ms/step - loss: 0.5172 - val_loss: 0.4811
Epoch 4/6
1210/1210 [==============================] - 3s 2ms/step - loss: 0.5158 - val_loss: 0.4867
Epoch 5/6
1210/1210 [==============================] - 3s 2ms/step - loss: 0.5156 - val_loss: 0.4859
Epoch 6/6
135/135 [==============================] - 0s 763us/step
-0.7219475085583872 -0.8823802882380288 0.7941422594142259
Current x =  [[553.32536889 718.03747271 606.73574231 549.43435117 429.41825135]
 [269.1015625  586.2109375  539.8046875  199.4921875  222.6953125 ]
 [270.55175781 958.91113281 364.33105469 5

Iteration= 34  x_best= [257.5 752.5 257.5 752.5 257.5] y= [-0.78580873]
C = [427.89794922 833.46923828 766.76025391 542.46337891 160.57861328]
Epoch 1/6
1210/1210 [==============================] - 4s 3ms/step - loss: 0.5466 - val_loss: 0.4842
Epoch 2/6
1210/1210 [==============================] - 4s 3ms/step - loss: 0.5068 - val_loss: 0.5025
Epoch 3/6
1210/1210 [==============================] - 4s 3ms/step - loss: 0.5038 - val_loss: 0.4833
Epoch 4/6
1210/1210 [==============================] - 4s 3ms/step - loss: 0.5020 - val_loss: 0.4862
Epoch 5/6
1210/1210 [==============================] - 4s 3ms/step - loss: 0.5013 - val_loss: 0.4904
Epoch 6/6
135/135 [==============================] - 0s 799us/step
-0.7131696428571429 -0.891213389121339 0.7923124612523248
Current x =  [[553.32536889 718.03747271 606.73574231 549.43435117 429.41825135]
 [269.1015625  586.2109375  539.8046875  199.4921875  222.6953125 ]
 [270.55175781 958.91113281 364.33105469 592.49511719 673.70605469]
 [865.1318

Iteration= 37  x_best= [919.75585938 991.29882812 215.92773438 583.31054688 825.00976562] y= [-0.78518519]
C = [518.77685547 649.77783203 374.24072266 594.67041016 866.34033203]
Epoch 1/6
1210/1210 [==============================] - 3s 2ms/step - loss: 0.5776 - val_loss: 0.5101
Epoch 2/6
1210/1210 [==============================] - 2s 2ms/step - loss: 0.5244 - val_loss: 0.4983
Epoch 3/6
1210/1210 [==============================] - 2s 2ms/step - loss: 0.5213 - val_loss: 0.4845
Epoch 4/6
1210/1210 [==============================] - 2s 2ms/step - loss: 0.5189 - val_loss: 0.5053
Epoch 5/6
1210/1210 [==============================] - 2s 2ms/step - loss: 0.5186 - val_loss: 0.5064
Epoch 6/6
135/135 [==============================] - 0s 726us/step
-0.7650273224043715 -0.7810320781032078 0.7729468599033816
Current x =  [[553.32536889 718.03747271 606.73574231 549.43435117 429.41825135]
 [269.1015625  586.2109375  539.8046875  199.4921875  222.6953125 ]
 [270.55175781 958.91113281 364.33105469 5

## Evaluate results

As you are doing a real world experiment you do not have access to the real pareto front, then you fo noy hsbr yhr metrics as before (I removed them).

What you should evaluate is the results in terms of the best *y* possible. You have the data below, but you may do a pandas dataframe to store it cleaner when it is computed in the previous section.

GP.X === samples where the function was evaluated

GP.Y === results of those evaluations

In [6]:
plt.scatter(-GP.Y[:,0],-GP.Y[:,1])

IndexError: index 1 is out of bounds for axis 1 with size 1

In [ ]:
-GP.Y

In [ ]:
from utils.calc_pareto import get_pareto_undominated_by

front = get_pareto_undominated_by(results)


plt.scatter(-front[:,0],-front[:,1])

In [ ]:
-results

In [ ]:
x_front = [GP.X[(-GP.Y).tolist().index(y)] for y in front.tolist()]

In [ ]:
!mkdir Checkpoints

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score

front_test = []

data = pd.read_csv("heart_disease_health_indicators_BRFSS2015.csv")
X = data.drop("HeartDiseaseorAttack", axis=1)
y = data["HeartDiseaseorAttack"]

y_1 = y[y == 1]
y_used = pd.concat([y_1, y[y == 0].sample(len(y_1))])
X_used = X.iloc[y_used.index]

X_train_val, X_test, y_train_val, y_test = train_test_split(X_used, y_used, test_size=0.1, random_state=42, stratify=y_used)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1, random_state=42, stratify=y_train_val)


for arch in x_front:
    model = tf.keras.models.Sequential([
      tf.keras.layers.Flatten(input_shape=(21, 1)),
      tf.keras.layers.Dense(int(np.around(arch[0])), activation='relu'),
      tf.keras.layers.Dense(int(np.around(arch[1])), activation='relu'),
      tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    
    checkpoint_path = f"./Checkpoints/model_{int(np.around(arch[0]))}_{int(np.around(arch[1]))}"
    checkpoint_dir = os.path.dirname(checkpoint_path)

    
    model.load_weights(checkpoint_path)
    
    y_pred = np.around(model.predict(X_test))
    
    front_test.append((precision_score(y_test, y_pred), recall_score(y_test, y_pred)))
    

In [ ]:
[x[0] for x in front_test]

plt.scatter([x[0] for x in front_test], [x[1] for x in front_test])

In [ ]:
front_test

In [ ]:
#!rm -rf Checkpoints